In [ ]:
! ls -l /kaggle/input/
! ls -l /kaggle/input/plant-pathology-2021-fgvc8
! ls -l /kaggle/input/plant-pathology-model-zoo

In [ ]:
! pip install -q --no-index /kaggle/input/plant-pathology-model-zoo/*.whl
! pip install -q --no-index /kaggle/input/plant-pathology-model-zoo/kaggle_plant_pathology-0.4.1-*
! pip list | grep torch
! pip list | grep kornia
! pip list | grep kaggle

In [ ]:
import torch
from kaggle_plantpatho.data import PlantPathologyDM
from kaggle_plantpatho.models import LitResnet, MultiPlantPathology

dm = PlantPathologyDM(
    base_path='/kaggle/input/plant-pathology-2021-fgvc8',
    path_csv='/kaggle/input/plant-pathology-2021-fgvc8/train.csv',
    simple=False,
    batch_size=32,
)
dm.setup()

net = torch.load('/kaggle/input/plant-pathology-model-zoo/fgvc8_resnet50.pt')
# net = torch.load('/kaggle/input/plant-pathology-model-zoo/fgvc8_resnext101_32x8d.pt')
model = MultiPlantPathology(model=net)
# ckpt = torch.load('/kaggle/input/plant-pathology-model-zoo/resnet50-epoch38-valid_acc0.9678-valid_f10.9028.ckpt', map_location=torch.device('cpu'))
# model.load_state_dict(ckpt['state_dict'])

In [ ]:
import numpy as np
import pandas as pd

preds = []
model.cuda().eval()

for imgs, names in dm.test_dataloader():
    with torch.no_grad():
        onehots = model(imgs.cuda()).cpu()
    print(np.round(onehots.detach().numpy(), decimals=2))
    for oh, name in zip(onehots, names):
        print(dm.labels_unique[torch.argmax(oh)])
        lbs = dm.onehot_to_labels(oh)
        preds.append(dict(image=name, labels=" ".join(lbs[::-1])))

df_preds = pd.DataFrame(preds)
print(df_preds)

In [ ]:
df_preds['labels'] = [lbs or "healthy" for lbs in df_preds['labels']]
print(df_preds)
df_preds.to_csv('submission.csv', index=False)